In [1]:
import torch
import os

In [15]:
def do_swa(checkpoint):
	skip = ['relative_position_index', 'num_batches_tracked']

	K = len(checkpoint)
	swa = None
	
	for k in range(K):
		state_dict = torch.load(checkpoint[k], map_location=lambda storage, loc: storage) # ['model_state_dict']
		if swa is None:
			swa = state_dict
		else:
			for k, v in state_dict.items():
				# print(k)
				if any(s in k for s in skip): continue
				swa[k] += v
	
	for k, v in swa.items():
		if any(s in k for s in skip): continue
		swa[k] /= K
	
	return swa

In [25]:
def do_swa_yolo(checkpoint):
	skip = ['relative_position_index', 'num_batches_tracked']

	K = len(checkpoint)
	swa = None
	
	for k in range(K):
		state_dict = torch.load(checkpoint[k], map_location=lambda storage, loc: storage)['optimizer']['state'] # ['model_state_dict']
		if swa is None:
			swa = state_dict
		else:
			for k, v in state_dict.items():
				# print(k)
				#if any(s in k for s in skip): continue
				swa[k]['momentum_buffer'] += v['momentum_buffer']
	
	for k, v in swa.items():
		# if any(s in k for s in skip): continue
		swa[k]['momentum_buffer'] /= K
	
	return swa

In [45]:
checkpoints = ['../weights/crnn_combined_after_filtering_and_iqgj_efficientnet/e601_0.0302_crnn_combined_after_filtering_and_iqgj_efficientnet.pth',
               '../weights/crnn_combined_after_filtering_and_iqgj_efficientnet/e701_0.0236_crnn_combined_after_filtering_and_iqgj_efficientnet.pth',
               '../weights/crnn_combined_after_filtering_and_iqgj_efficientnet/e801_0.0216_crnn_combined_after_filtering_and_iqgj_efficientnet.pth',
               '../weights/crnn_combined_after_filtering_and_iqgj_efficientnet/e901_0.0193_crnn_combined_after_filtering_and_iqgj_efficientnet.pth']

In [29]:
checkpoints = ['/media/kudaibergen/TS512/projects_ts/teye-sa-debugger-python/models/SA_80cls_codetr_coco_relabeled_no_crowds_from_pretrained_custom_hypers4/epoch70.pt',
               '/media/kudaibergen/TS512/projects_ts/teye-sa-debugger-python/models/SA_80cls_codetr_coco_relabeled_no_crowds_from_pretrained_custom_hypers4/epoch80.pt',
               '/media/kudaibergen/TS512/projects_ts/teye-sa-debugger-python/models/SA_80cls_codetr_coco_relabeled_no_crowds_from_pretrained_custom_hypers4/epoch90.pt',
]

In [43]:
ROOT = '../weights/crnn_combined_after_filtering_and_iqgj_efficientnet/'
# checkpoints = os.listdir(ROOT)

In [46]:
checkpoints = sorted(checkpoints, key=lambda x: int(x.split('_')[0][1:]))

ValueError: invalid literal for int() with base 10: './weights/crnn'

In [5]:
checkpoints = [ROOT + i for i in checkpoints]

In [35]:
swa_start_checkpoint = 240

In [33]:
len(checkpoints)

298

In [38]:
checkpoints[swa_start_checkpoint:swa_start_checkpoint+20]

['../weights/crnn_combined_after_filtering_and_iqgj_efficientnet/e793_0.0254_crnn_combined_after_filtering_and_iqgj_efficientnet.pth',
 '../weights/crnn_combined_after_filtering_and_iqgj_efficientnet/e794_0.0247_crnn_combined_after_filtering_and_iqgj_efficientnet.pth',
 '../weights/crnn_combined_after_filtering_and_iqgj_efficientnet/e795_0.0204_crnn_combined_after_filtering_and_iqgj_efficientnet.pth',
 '../weights/crnn_combined_after_filtering_and_iqgj_efficientnet/e801_0.0216_crnn_combined_after_filtering_and_iqgj_efficientnet.pth',
 '../weights/crnn_combined_after_filtering_and_iqgj_efficientnet/e802_0.0213_crnn_combined_after_filtering_and_iqgj_efficientnet.pth',
 '../weights/crnn_combined_after_filtering_and_iqgj_efficientnet/e803_0.0191_crnn_combined_after_filtering_and_iqgj_efficientnet.pth',
 '../weights/crnn_combined_after_filtering_and_iqgj_efficientnet/e811_0.0235_crnn_combined_after_filtering_and_iqgj_efficientnet.pth',
 '../weights/crnn_combined_after_filtering_and_iqgj_eff

In [32]:
for_swa_save = torch.load(checkpoints[0]) #['optimizer']['state'][0]['momentum_buffer']

In [30]:
swa_state_dict = do_swa_yolo(checkpoints) #checkpoints[swa_start_checkpoint:swa_start_checkpoint+20])

In [33]:
for_swa_save['optimizer']['state'] = swa_state_dict

In [34]:
torch.save(for_swa_save, '/media/kudaibergen/TS512/projects_ts/teye-sa-debugger-python/models/swa.pt')

In [29]:
checkpoints[swa_start_checkpoint], \
checkpoints[-1]

('../weights/crnn_combined_after_filtering_and_iqgj_efficientnet/e672_0.0217_crnn_combined_after_filtering_and_iqgj_efficientnet.pth',
 '../weights/crnn_combined_after_filtering_and_iqgj_efficientnet/e973_0.0193_crnn_combined_after_filtering_and_iqgj_efficientnet.pth')

In [40]:
save_file_name = '../weights/crnn_combined_after_filtering_and_iqgj_efficientnet/swa_crnn_combined_after_filtering_and_iqgj_efficientnet.pth'

In [41]:
torch.save({'model_state_dict':swa_state_dict}, save_file_name)

In [24]:
checkpoints[swa_start_checkpoint:]

['../weights/crnn_combined_after_filtering_and_iqgj_efficientnet/e927_0.0212_crnn_combined_after_filtering_and_iqgj_efficientnet.pth',
 '../weights/crnn_combined_after_filtering_and_iqgj_efficientnet/e931_0.0261_crnn_combined_after_filtering_and_iqgj_efficientnet.pth',
 '../weights/crnn_combined_after_filtering_and_iqgj_efficientnet/e934_0.0251_crnn_combined_after_filtering_and_iqgj_efficientnet.pth',
 '../weights/crnn_combined_after_filtering_and_iqgj_efficientnet/e936_0.0223_crnn_combined_after_filtering_and_iqgj_efficientnet.pth',
 '../weights/crnn_combined_after_filtering_and_iqgj_efficientnet/e939_0.0197_crnn_combined_after_filtering_and_iqgj_efficientnet.pth',
 '../weights/crnn_combined_after_filtering_and_iqgj_efficientnet/e941_0.0306_crnn_combined_after_filtering_and_iqgj_efficientnet.pth',
 '../weights/crnn_combined_after_filtering_and_iqgj_efficientnet/e942_0.0275_crnn_combined_after_filtering_and_iqgj_efficientnet.pth',
 '../weights/crnn_combined_after_filtering_and_iqgj_eff